In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import sys
from copy import deepcopy
from typing import List, Dict, Tuple, Optional, Union, Callable
import pandas as pd
import re
from pathlib import Path
from datetime import datetime
import plotly.express as px

def find_csv_files(directory: str):
    directory_path = Path(directory) # Convert string path to a Path object
    return list(directory_path.glob('**/*.csv')) # Return a list of all .csv files in the directory and its subdirectories

# found_session_export_paths = [Path(v).resolve() for v in  ["C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# ]]

def parse_filename(path: Path, debug_print:bool=False) -> Tuple[datetime, str, str]:
    """ 
    # # from the found_session_export_paths, get the most recently exported laps_csv, ripple_csv (by comparing `export_datetime`) for each session (`session_str`)
    # a_export_filename: str = "2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv"
    # export_datetime = "2024-01-12_0420PM"
    # session_str = "kdiba_pin01_one_fet11-01_12-58-54"
    # export_file_type = "(laps_marginals_df)" # .csv

    # # return laps_csv, ripple_csv
    # laps_csv = Path("C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv").resolve()
    # ripple_csv = Path("C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv").resolve()

    """
    filename = path.stem   # Get filename without extension
    
    pattern = r"(.*_\d{2}\d{2}[APMF]{2})-(.*)-(\(.+\))"
    match = re.match(pattern, filename)
    
    if match is None:
        if debug_print:
            print(f'did not match pattern with time.')
        # day_date_only_pattern = r"(.*(?:_\d{2}\d{2}[APMF]{2})?)-(.*)-(\(.+\))"
        day_date_only_pattern = r"(\d{4}-\d{2}-\d{2})-(.*)-(\(.+\))" # 
        day_date_only_match = re.match(day_date_only_pattern, filename) # '2024-01-04-kdiba_gor01_one_2006-6-08_14-26'        
        if day_date_only_match is None:
            print(f'ERR: Could not parse filename: "{filename}"') # 2024-01-18_GL_t_split_df
            # raise ValueError(f'Could not parse filename: {filename}')
            return None, None, None # used to return ValueError when it couldn't parse, but we'd rather skip unparsable files
        
        export_datetime_str, session_str, export_file_type = day_date_only_match.groups()
        # print(export_datetime_str, session_str, export_file_type)
        # parse the datetime from the export_datetime_str and convert it to datetime object
        export_datetime = datetime.strptime(export_datetime_str, "%Y-%m-%d")
    else:
        export_datetime_str, session_str, export_file_type = match.groups()
        # parse the datetime from the export_datetime_str and convert it to datetime object
        export_datetime = datetime.strptime(export_datetime_str, "%Y-%m-%d_%I%M%p")
        
    # Trim the brackets from the file type
    export_file_type = export_file_type[1:-1]
    return export_datetime, session_str, export_file_type


debug_print: bool = False

Automatic pdb calling has been turned OFF


In [2]:
## Load across session t_delta CSV, which contains the t_delta for each session:
# t_split_csv_path = Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\output\collected_outputs\2024-01-18_GL_t_split_df.csv').resolve() # Apogee
t_split_csv_path = Path('/home/halechr/cloud/turbo/Data/Output/collected_outputs/2024-01-18_GL_t_split_df.csv').resolve() # GL

t_split_df = pd.read_csv(t_split_csv_path, index_col=0) # Assuming that your CSV file has an index column
t_split_dict = t_split_df.to_dict(orient='index')
t_split_df

,t_start,t_delta,t_end
kdiba_gor01_one_2006-6-08_14-26-15,0.0,1211.558080,2093.897857
kdiba_gor01_one_2006-6-09_1-22-43,0.0,1029.316609,1737.196831
kdiba_gor01_one_2006-6-12_15-55-31,0.0,656.064809,1122.186487
kdiba_gor01_two_2006-6-07_16-40-19,0.0,1236.266245,2587.801682
kdiba_gor01_two_2006-6-08_21-16-25,0.0,722.653951,1201.083936
kdiba_gor01_two_2006-6-09_22-24-40,0.0,911.601160,2573.457162
kdiba_gor01_two_2006-6-12_16-53-46,0.0,471.067400,785.451326
kdiba_vvp01_one_2006-4-09_17-29-30,0.0,873.624498,1391.655628
kdiba_vvp01_one_2006-4-10_12-25-50,0.0,883.897132,1413.399172
kdiba_vvp01_two_2006-4-09_16-40-54,0.0,1155.706469,1724.033140


In [ ]:
# t_split_df[t_split_df.index == 'kdiba_gor01_one_2006-6-08_14-26-15']

# t_split_df.to_dict(orient='list')

t_split_dict.get('kdiba_gor01_one_2006-6-08_14-26-15', {}).get('t_delta', None)

In [3]:

## BEGIN
# csv_files = find_csv_files("C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs") # Apogee
csv_files = find_csv_files('/home/halechr/cloud/turbo/Data/Output/collected_outputs') # GL

found_session_export_paths = csv_files
found_session_export_paths

# parse all filenames and store in a list of tuples
parsed_paths = [(*parse_filename(p), p) for p in found_session_export_paths if (parse_filename(p)[0] is not None)]
# sort the list by datetime, descending
parsed_paths.sort(reverse=True)

if debug_print:
    print(f'parsed_paths: {parsed_paths}')

sessions = {}
for export_datetime, session_str, file_type, path in parsed_paths:
    if session_str not in sessions:
        sessions[session_str] = {}
    
    # if this file type hasn't been set for this session, or if this file is more recent, set this file as the most recent
    if (file_type not in sessions[session_str]) or (sessions[session_str][file_type][-1] < export_datetime):
        sessions[session_str][file_type] = (path, export_datetime)
        
# now sessions is a dictionary where the key is the session_str and the value is another dictionary.
# This inner dictionary's key is the file type and the value is the most recent path for this combination of session and file type
# Thus, laps_csv and ripple_csv can be obtained from the dictionary for each session

final_sessions = {}
final_sessions_loaded_laps_dict = {}
final_sessions_loaded_ripple_dict = {}
for session_str, session_dict in sessions.items():
	final_sessions[session_str] = {}
	for file_type, (a_path, an_export_datetime) in session_dict.items():
		final_sessions[session_str][file_type] = a_path
		
	session_name = str(session_str)  # Extract session name from the filename
	if debug_print:
		print(f'processing session_name: {session_name}')

	laps_file = final_sessions[session_str]['laps_marginals_df']
	ripple_file = final_sessions[session_str]['ripple_marginals_df']

	curr_session_t_split = t_split_dict.get('kdiba_gor01_one_2006-6-08_14-26-15', {}).get('t_delta', None)
	if curr_session_t_split is None:
		print(f'WARN: curr_session_t_split is None for session_str = "{session_str}"')
	laps_df = pd.read_csv(laps_file)
	laps_df['session_name'] = session_name # fixed session name for all rows in df
	if curr_session_t_split is not None:
		# compute the time aligned to the session's t_delta:
		laps_df['delta_aligned_start_t'] = laps_df['lap_start_t'] - curr_session_t_split
	final_sessions_loaded_laps_dict[session_str] = laps_df

	ripple_df = pd.read_csv(ripple_file)
	ripple_df['session_name'] = session_name # fixed session name for all rows in df
	if curr_session_t_split is not None:
		# compute the time aligned to the session's t_delta:
		ripple_df['delta_aligned_start_t'] = ripple_df['ripple_start_t'] - curr_session_t_split
	final_sessions_loaded_ripple_dict[session_str] = ripple_df


# final_sessions
# {'kdiba_gor01_one_2006-6-08_14-26-15': {'ripple_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-17_0540PM-kdiba_gor01_one_2006-6-08_14-26-15-(ripple_marginals_df).csv'),
#   'laps_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-17_0540PM-kdiba_gor01_one_2006-6-08_14-26-15-(laps_marginals_df).csv')},
#  'kdiba_gor01_one_2006-6-09_1-22-43': {'ripple_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0838PM-kdiba_gor01_one_2006-6-09_1-22-43-(ripple_marginals_df).csv'),
#   'laps_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0838PM-kdiba_gor01_one_2006-6-09_1-22-43-(laps_marginals_df).csv')},
#  'kdiba_pin01_one_fet11-01_12-58-54': {'ripple_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv'),
#   'laps_marginals_df': WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv')}}

all_sessions_laps_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_laps_dict.values()), axis='index', ignore_index=True)
all_sessions_ripple_df: pd.DataFrame = pd.concat(list(final_sessions_loaded_ripple_dict.values()), axis='index', ignore_index=True)
all_sessions_laps_df

ERR: Could not parse filename: "2024-01-03_GL_527am-2006-4-10_12-58-3_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-2006-6-12_16-53-46_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-2006-4-09_16-40-54_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-02_GL_831pm-2006-6-12_16-53-46_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-fet11-01_12-58-54_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-18_GL_t_split_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-2006-4-09_17-29-30_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-02_GL_831pm-11-03_12-3-25_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-03_GL_527am-2006-6-09_1-22-43_merged_complete_epoch_stats_df"
ERR: Could not parse filename: "2024-01-02_GL_831pm-2006-6-07_16-40-19_merged_complete_epoch_stats_df"
ERR: Could not parse fi

,Unnamed: 0,P_LR,P_RL,P_Long,P_Short,lap_idx,lap_start_t,session_name,delta_aligned_start_t
0,0,0.407431,0.592569,0.515200,0.484800,0,36.559134,kdiba_pin01_one_fet11-01_12-58-54,-1174.998946
1,1,0.619211,0.380789,0.469827,0.530173,1,85.804912,kdiba_pin01_one_fet11-01_12-58-54,-1125.753168
2,2,0.519518,0.480482,0.747540,0.252460,2,111.466570,kdiba_pin01_one_fet11-01_12-58-54,-1100.091510
3,3,0.532311,0.467689,0.740956,0.259044,3,145.699247,kdiba_pin01_one_fet11-01_12-58-54,-1065.858833
4,4,0.380348,0.619652,0.754087,0.245913,4,186.279645,kdiba_pin01_one_fet11-01_12-58-54,-1025.278435
...,...,...,...,...,...,...,...,...,...
1241,75,0.806312,0.193688,0.279490,0.720510,75,2002.883510,kdiba_gor01_one_2006-6-08_14-26-15,791.325430
1242,76,0.180555,0.819445,0.293255,0.706745,76,2019.131459,kdiba_gor01_one_2006-6-08_14-26-15,807.573379
1243,77,0.820589,0.179411,0.352401,0.647599,77,2026.006596,kdiba_gor01_one_2006-6-08_14-26-15,814.448516
1244,78,0.118055,0.881945,0.329319,0.670681,78,2037.617503,kdiba_gor01_one_2006-6-08_14-26-15,826.059423


In [5]:
def plot_across_sessions_results(directory, concatenated_laps_df, concatenated_ripple_df, save_figures=False, figure_save_extension='.png'):
    """ takes the directory containing the .csv pairs that were exported by `export_marginals_df_csv`
    Produces and then saves figures out the the f'{directory}/figures/' subfolder

    """
    if not isinstance(directory, Path):
        directory = Path(directory).resolve()
    assert directory.exists()
    print(f'plot_across_sessions_results(directory: {directory})')
    if save_figures:
        # Create a 'figures' subfolder if it doesn't exist
        figures_folder = Path(directory, 'figures')
        figures_folder.mkdir(parents=False, exist_ok=True)
        assert figures_folder.exists()
        print(f'\tfigures_folder: {figures_folder}')
    
    # Create an empty list to store the figures
    all_figures = []



    ## delta_t aligned:
    # Create a bubble chart for laps
    fig_laps = px.scatter(concatenated_laps_df, x='delta_aligned_start_t', y='P_Long', title=f"Laps - Session: {session_name}", color='session_name')
    # Create a bubble chart for ripples
    fig_ripples = px.scatter(concatenated_ripple_df, x='delta_aligned_start_t', y='P_Long', title=f"Ripples - Session: {session_name}", color='session_name')


    # # Create a bubble chart for laps
    # fig_laps = px.scatter(concatenated_laps_df, x='lap_start_t', y='P_Long', title=f"Laps - Session: {session_name}", color='session_name')
    # # Create a bubble chart for ripples
    # fig_ripples = px.scatter(concatenated_ripple_df, x='ripple_start_t', y='P_Long', title=f"Ripples - Session: {session_name}", color='session_name')

    if save_figures:
        # Save the figures to the 'figures' subfolder
        print(f'\tsaving figures...')
        fig_laps_name = Path(figures_folder, f"{session_name}_laps_marginal{figure_save_extension}").resolve()
        print(f'\tsaving "{fig_laps_name}"...')
        fig_laps.write_image(fig_laps_name)
        fig_ripple_name = Path(figures_folder, f"{session_name}_ripples_marginal{figure_save_extension}").resolve()
        print(f'\tsaving "{fig_ripple_name}"...')
        fig_ripples.write_image(fig_ripple_name)
    
    # Append both figures to the list
    all_figures.append((fig_laps, fig_ripples))
    
    return all_figures

# Example usage:
# directory = '/home/halechr/FastData/collected_outputs/'
# directory = r'C:\Users\pho\Desktop\collected_outputs'
# directory = r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs' # APOGEE
directory = '/home/halechr/cloud/turbo/Data/Output/collected_outputs' # GL
all_session_figures = plot_across_sessions_results(directory, concatenated_laps_df=all_sessions_laps_df, concatenated_ripple_df=all_sessions_ripple_df, save_figures=False)

# Show figures for all sessions
for fig_laps, fig_ripples in all_session_figures:
    fig_laps.show()
    fig_ripples.show()
    # fig_laps.write_html("../output/2024-01-18_AcrossSession_fig_laps.html")
    # fig_ripples.write_html("../output/2024-01-18_AcrossSession_fig_ripples.html")

    fig_laps.write_html("output/2024-01-18_AcrossSession_fig_laps.html")
    fig_ripples.write_html("output/2024-01-18_AcrossSession_fig_ripples.html")


plot_across_sessions_results(directory: /nfs/turbo/umms-kdiba/Data/Output/collected_outputs)
